<a href="https://colab.research.google.com/github/codewithjaspreet/NLP/blob/master/Naive_Bayes_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import nltk
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords

import numpy as np

In [57]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
pt = twitter_samples.strings('positive_tweets.json')
nt = twitter_samples.strings('negative_tweets.json')

In [59]:
np.random.seed(1)
rand = np.random.randint(0 , 5000 , 5)

In [60]:
print('Positive Tweets ::')
print('\033[92m')
for i in rand:
  print(pt[i])

Positive Tweets ::

@ZarlashtFaisal @Tabinda_Samar Sethi was HIGH ??? :)
Fav if awake fam :)
Just smile even your in Pain :) http://t.co/AxTiqf0xek
camillus pleaseee? :)
Why have i just painted my nails pink :) ???


In [61]:
print('Negative Tweets ::')
print('\033[92m')
for i in rand:
  print(nt[i])

Negative Tweets ::

I need a big cuddle from Lew and kisses on my face :(((( I don't want to go through this again
@kaiality too late now :(
traffic :-(
Soft defence by the best defensive team there :( #NRLTigersRoosters
@LukeBryanOnline Yayyyy!!! I hope it's not while I am knocked out by anesthesia. I will be so sad if I miss it :(


In [62]:
import re
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

def processTweet(tweet):
    stemmer  = PorterStemmer()
    stopwords_english = stopwords.words('english')


    # remove stock market tickers like $GE

    tweet = re.sub(r'\$\w*', '', tweet)

    # remove old style retweet text "RT"

    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks

    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # remove hashtags

    # only removing the hash # sign from the word

    tweet = re.sub(r'#', '', tweet)

    # tokenize tweets


    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,

                                 reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []

    # remove stopwords, and punctuations

    for wrd in tweet_tokens:
        if (wrd not in stopwords_english and  # remove stopwords
                wrd not in string.punctuation):

               stem  = stemmer.stem(wrd)
               tweets_clean.append(stem)                                     

    return tweets_clean

In [63]:
tweet = pt[np.random.randint(0,5000)]
print('Raw Tweet : \n ' , tweet)
tweet = processTweet(tweet)

print('After Processing : \n' , tweet )

Raw Tweet : 
  @WforWoman 5. Over 20 W kurtas! And my Mom has about half the number I have :D #WSaleLove
After Processing : 
 ['5', '20', 'w', 'kurta', 'mom', 'half', 'number', ':d', 'wsalelov']


In [64]:
def count_tweets(tweets, ys):
  ys_list = np.squeeze(ys).tolist()
  freqs ={}

  for y, tweet in zip(ys_list, tweets):
    for word in processTweet(tweet):
      pair = (word, y)
      if pair in freqs:
        freqs[pair] +=1
      else:
        freqs[pair] = 1
  
  return freqs

In [65]:
def lookup(freqs, word, label):
  n = 0
  pair = (word, label)
  if pair in freqs:
    n = freqs[pair]
  return n 

In [66]:
# splitting the data for training and testing 
train_pos = pt[:4000]
test_pos = pt[4000:]

train_neg = nt[:4000]
test_neg = nt[4000:]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# numpy array for the labels in the training set
train_y = np.append(np.ones((len(train_pos))), np.zeros((len(train_neg))))
test_y = np.append(np.ones((len(test_neg))), np.zeros((len(test_neg))))

In [67]:
# Build a frequency dictionary
freqs = count_tweets(train_x, train_y)

def train_naive_bayes(freqs, train_x, train_y):
  logliklihood = {}
  logprior = 0

  # calculate V, number of unique words in the vocabulary
  vocab = set([pair[0] for pair in freqs.keys()])
  V = len(vocab)

  ## Calculate N_pos, N_neg, V_pos, V_neg
  # N_pos : total number of positive words
  # N_neg : total number of negative words
  # V_pos : total number of unique positive words
  # V_neg : total number of unique negative words

  N_pos = N_neg = V_pos = V_neg = 0
  for pair in freqs.keys():
    if pair[1]>0:
      V_pos +=1
      N_pos += freqs[pair]
    else:
      V_neg +=1
      N_neg += freqs[pair]

  # Number of Documents (tweets)
  D = len(train_y)

  # D_pos, number of positive documnets
  D_pos = len(list(filter(lambda x: x>0, train_y)))

  # D_pos, number of negative documnets
  D_neg = len(list(filter(lambda x: x<=0, train_y)))

  # calculate the logprior
  logprior = np.log(D_pos) - np.log(D_neg)

  for word in vocab:
    freqs_pos = lookup(freqs, word, 1)
    freqs_neg = lookup(freqs, word, 0)

    # calculte the probability of each word being positive and negative
    p_w_pos = (freqs_pos+1)/(N_pos+V)
    p_w_neg = (freqs_neg+1)/(N_neg+V)


  return logprior,logliklihood



In [68]:

logprior ,loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

0.0
0


In [69]:

def naive_bayes_predict(tweet, logprior, loglikelihood):
  word_l = processTweet(tweet)
  p = 0
  p+=logprior

  for word in word_l:
    if word in loglikelihood:
      p+=loglikelihood[word]

  return p

In [70]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
  accuracy = 0
  y_hats = []
  for tweet in test_x:
    if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
      y_hat_i = 1
    else:
      y_hat_i = 0
    y_hats.append(y_hat_i)
  error = np.mean(np.absolute(test_y - y_hats))
  accuracy = 1-error

  return accuracy
  
  print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

In [71]:
tweets = ['I am happy' , 'I am bad', 'this movie should have been great. ', 'great', 'great great', 'great great great', 'great great great great', 'I am not happy : (' ]
for tweet in tweets:
  p = naive_bayes_predict(tweet, logprior, loglikelihood)

  if p>1:
    print ('\033 [92m' )
    print (f' {tweet} : : Positive sentiment ({p:.2f}) ')

  else:
    print ('\033[91m' )
    print (f' {tweet} : : Negative sentiment ({p:.2f}) ')



 I am happy : : Negative sentiment (0.00) 

 I am bad : : Negative sentiment (0.00) 

 this movie should have been great.  : : Negative sentiment (0.00) 

 great : : Negative sentiment (0.00) 

 great great : : Negative sentiment (0.00) 

 great great great : : Negative sentiment (0.00) 

 great great great great : : Negative sentiment (0.00) 

 I am not happy : ( : : Negative sentiment (0.00) 
